### Import Necessary Packages

In [ ]:
import boto3 
import gc
import numpy as np
import sagemaker 
from sagemaker import get_execution_role
import os

### Define Bucket and Prefix for Training and Validation data 

In [ ]:
bucket = sagemaker.Session().default_bucket()
prefix = 'demand-prediction'

### Configure Necessary Parameters 

In [ ]:
algo_name = 'lightgbm'
ecr_repository_name = algo_name 
role = get_execution_role()
account_id = role.split(':')[4]
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
print(account_id)
print(region)

### Build and Push Docker Image to ECR 

In [ ]:
%%bash -s "$account_id" "$region"
cd light_gbm_sagemaker/script 
./build_and_push.sh $1 $2 lightgbm

### Train Model by the Docker Image Just Pushed to ECR

In [ ]:
container_image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest'.format(account_id, region, ecr_repository_name)
print(container_image_uri)


In [ ]:
import sagemaker
import time

job_name = "{}-{}".format(algo_name, str(time.time()))
est = sagemaker.estimator.Estimator(container_image_uri,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m5.xlarge',
                                    base_job_name=prefix)

est.set_hyperparameters(num_iterations=100)
train_config = sagemaker.session.s3_input('s3://{0}/{1}/train/'.format(bucket, prefix), content_type='text/csv')
val_config = sagemaker.session.s3_input('s3://{0}/{1}/validation/'.format(bucket, prefix), content_type='text/csv')

est.fit({'train': train_config, 'valid': val_config })
